# เขียน CSV ลง S3 (raw bucket)

In [5]:
import boto3
import sagemaker
import pandas as pd

# สร้าง SageMaker session และดึงข้อมูลพื้นฐาน
sess = sagemaker.Session()
bucket = sess.default_bucket()               # ใช้ default bucket ของ SageMaker ใน account/region นี้
role = sagemaker.get_execution_role()
region = boto3.Session().region_name

print("SageMaker default bucket:", bucket)
print("Region:", region)

# ระบุ path ใน S3 สำหรับเก็บไฟล์ CSV (raw zone)
s3_private_path_csv = "s3://{}/retail-demand-forecasting/csv/".format(bucket)
print("S3 raw path for CSV:", s3_private_path_csv)

# เก็บตัวแปรนี้ด้วย %store เพื่อใช้ในโน้ตบุ๊กถัดไป (เช่น Athena, Clarify, ฯลฯ)
%store s3_private_path_csv

# ระบุ path ของไฟล์ CSV ใน local (ในโฟลเดอร์ data)
local_csv_path = "./data/retail-demand-forecasting.csv"

# อัปโหลดไฟล์จาก local -> S3
!aws s3 cp $local_csv_path $s3_private_path_csv

# ตรวจสอบว่าไฟล์ถูกอัปโหลดขึ้น S3 แล้ว
!aws s3 ls $s3_private_path_csv


SageMaker default bucket: sagemaker-us-east-1-423623839320
Region: us-east-1
S3 raw path for CSV: s3://sagemaker-us-east-1-423623839320/retail-demand-forecasting/csv/
Stored 's3_private_path_csv' (str)
upload: data/retail-demand-forecasting.csv to s3://sagemaker-us-east-1-423623839320/retail-demand-forecasting/csv/retail-demand-forecasting.csv
2025-12-01 07:07:27      37284 retail-demand-forecasting.csv


# สร้าง Athena Database

In [6]:
from pyathena import connect
import pandas as pd
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

database_name = "retail_demand"  # ตั้งชื่อ DB ที่จะใช้กับโปรเจ็คนี้

# ที่เก็บผล temporary ของ Athena
s3_staging_dir = f"s3://{bucket}/athena/staging"

conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# สร้าง Database (ถ้ายังไม่มี)
statement = f"CREATE DATABASE IF NOT EXISTS {database_name}"
pd.read_sql(statement, conn)

# ตรวจสอบว่า DB ถูกสร้างแล้ว
statement = "SHOW DATABASES"
df_show = pd.read_sql(statement, conn)
df_show.head()


/tmp/ipykernel_4012/1344931186.py:19: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)
/tmp/ipykernel_4012/1344931186.py:23: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_show = pd.read_sql(statement, conn)


,database_name
0,default
1,dsoaws
2,retail_demand
3,sagemaker_featurestore


# Register CSV Data กับ Athena (สร้าง External Table)

In [7]:
from pyathena import connect
import pandas as pd
import boto3
import sagemaker

sess = sagemaker.Session()
bucket = sess.default_bucket()
region = boto3.Session().region_name

# ดึง path S3 ที่เรา %store ไว้
%store -r s3_private_path_csv
print("Using S3 path:", s3_private_path_csv)

database_name = "retail_demand"
table_name_tsv = "demand_product"   # ชื่อตารางใน Athena

s3_staging_dir = f"s3://{bucket}/athena/staging"
conn = connect(region_name=region, s3_staging_dir=s3_staging_dir)

# สร้าง External Table ให้ตรงกับ CSV schema
statement = f"""
CREATE EXTERNAL TABLE IF NOT EXISTS {database_name}.{table_name_tsv} (
    record_id       BIGINT,
    date            STRING,
    store_id        BIGINT,
    day_of_week     STRING,
    is_weekend      BIGINT,
    is_holiday      BIGINT,
    holiday_name    STRING,
    max_temp_c      DOUBLE,
    rainfall_mm     DOUBLE,
    is_hot_day      BIGINT,
    is_rainy_day    BIGINT,
    base_price      DOUBLE,
    discount_pct    DOUBLE,
    is_promo        BIGINT,
    promo_type      STRING,
    final_price     DOUBLE,
    units_sold      BIGINT
)
ROW FORMAT SERDE 'org.apache.hadoop.hive.serde2.lazy.LazySimpleSerDe'
WITH SERDEPROPERTIES (
  'serialization.format' = ',',
  'field.delim' = ','
)
STORED AS TEXTFILE
LOCATION '{s3_private_path_csv}'
TBLPROPERTIES ('skip.header.line.count'='1')
"""

pd.read_sql(statement, conn)
print("Created/updated table:", f"{database_name}.{table_name_tsv}")


Using S3 path: s3://sagemaker-us-east-1-423623839320/retail-demand-forecasting/csv/


/tmp/ipykernel_4012/2257607858.py:51: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(statement, conn)


Created/updated table: retail_demand.demand_product


## ตรวจสอบว่า Table ใช้งานได้จริง

In [8]:
statement = f"SHOW TABLES IN {database_name}"
df_tables = pd.read_sql(statement, conn)
df_tables.head()


/tmp/ipykernel_4012/3575404229.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_tables = pd.read_sql(statement, conn)


,tab_name
0,demand_product


In [9]:
statement = f"SELECT * FROM {database_name}.{table_name_tsv} LIMIT 10"
df_preview = pd.read_sql(statement, conn)
df_preview.head()


/tmp/ipykernel_4012/3997654001.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  df_preview = pd.read_sql(statement, conn)


,record_id,date,store_id,day_of_week,is_weekend,is_holiday,holiday_name,max_temp_c,rainfall_mm,is_hot_day,is_rainy_day,base_price,discount_pct,is_promo,promo_type,final_price,units_sold
0,1,2024-01-01,1,Monday,0,1,New Year,29.0,2.4,0,0,12.99,0.0,0,none,12.99,102
1,2,2024-01-02,1,Tuesday,0,0,None,27.5,9.3,0,1,14.85,0.0,0,none,14.85,82
2,3,2024-01-03,1,Wednesday,0,0,None,27.1,5.2,0,1,10.92,0.0,0,none,10.92,78
3,4,2024-01-04,1,Thursday,0,0,None,24.2,0.0,0,0,10.70,0.0,0,none,10.70,76
4,5,2024-01-05,1,Friday,0,0,None,26.2,0.0,0,0,13.93,0.0,0,none,13.93,81
